In [40]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

tf.__version__

'2.10.0'

In [41]:
# from google.colab import drive
# drive.mount('/content/drive')

# 당뇨병을 진단해봅시다.

In [42]:
xy = np.loadtxt('data-03-diabetes.csv',
                delimiter=',',
                dtype=np.float32)
x_train = xy[0:-100, 0:-1]
y_train = xy[0:-100, [-1]]

x_test = xy[-100:, 0:-1]
y_test = xy[-100:, [-1]]

print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(659, 8) (659, 1)
(100, 8) (100, 1)


In [43]:
print(x_test[0])
print(y_test[0])

[-0.294118  -0.0150754 -0.0491803 -0.333333  -0.550827   0.0134128
 -0.699402  -0.266667 ]
[1.]


In [44]:
dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(len(x_train))


In [45]:
W = tf.Variable(tf.random.normal([8,1],0,1), name='weight')
b = tf.Variable(tf.random.normal([1],0,1), name='bias')
tf.print(W,b)

[[-1.35441589]
 [0.704549313]
 [0.0366619118]
 ...
 [-0.534398437]
 [-0.0771029219]
 [1.5658046]] [-1.35441589]


## 가설 설정
* 병이 있다 / 없다로 분류
* sigmoid classification으로 진행


## $$ y_k = \frac{1}{1+e^{-x}}   $$

In [46]:
def logistic_regression(features):
    hypothesis  = tf.divide(1., 1. + tf.exp(-(tf.matmul(features, W) + b)))
    return hypothesis

print(logistic_regression(x_train))

tf.Tensor(
[[0.2946993 ]
 [0.18346919]
 [0.18662302]
 [0.29339463]
 [0.05839131]
 [0.171861  ]
 [0.07658403]
 [0.07001933]
 [0.65639555]
 [0.29951617]
 [0.16310543]
 [0.11796258]
 [0.28924388]
 [0.7262005 ]
 [0.11955827]
 [0.06620099]
 [0.12239756]
 [0.15031712]
 [0.1562363 ]
 [0.12712587]
 [0.21594909]
 [0.27217636]
 [0.07242237]
 [0.07219157]
 [0.22873715]
 [0.08535351]
 [0.11541592]
 [0.21731302]
 [0.34148768]
 [0.16527294]
 [0.05072254]
 [0.12969016]
 [0.12801765]
 [0.12886031]
 [0.09568814]
 [0.13716684]
 [0.14294603]
 [0.34933448]
 [0.12456456]
 [0.15914686]
 [0.17933565]
 [0.17752667]
 [0.24869716]
 [0.07587211]
 [0.29423264]
 [0.0918109 ]
 [0.07816202]
 [0.08139642]
 [0.08386741]
 [0.09359221]
 [0.06411801]
 [0.33162904]
 [0.19410497]
 [0.30720082]
 [0.2105107 ]
 [0.06234422]
 [0.17892934]
 [0.03477164]
 [0.40945533]
 [0.17045146]
 [0.14279957]
 [0.13613714]
 [0.19745669]
 [0.1152852 ]
 [0.09542069]
 [0.4513173 ]
 [0.09089641]
 [0.10599406]
 [0.08784565]
 [0.09674215]
 [0.07949

## Loss Function

* 기존 MSE 대신 Cross Entropy 사용

## $$
\begin{align}
loss(h(x),y) & = −y log(h(x))−(1−y)log(1−h(x))
\end{align}
$$

In [47]:
def loss_fn(hypothesis, labels):
    cost = -tf.reduce_mean(labels * tf.math.log(hypothesis) + \
                           (1 - labels) * tf.math.log(1 - hypothesis))
    return cost

optimizer = tf.compat.v1.train.GradientDescentOptimizer(learning_rate=0.005)

## 학습

In [54]:
epochs = 5000

for step in range(epochs):
  for features, labels in dataset:
    with tf.GradientTape() as tape:
        hypothesis = logistic_regression(features)
        loss_value = loss_fn(hypothesis,labels)
    grads = tape.gradient(loss_value, [W,b])
    optimizer.apply_gradients(grads_and_vars=zip(grads,[W,b]))
    if step % 100 == 0:
        print("Iter: {}, Loss: {:.4f}".format(step, loss_fn(logistic_regression(features),labels)))

Iter: 0, Loss: 0.5061
Iter: 100, Loss: 0.5057
Iter: 200, Loss: 0.5052
Iter: 300, Loss: 0.5047
Iter: 400, Loss: 0.5043
Iter: 500, Loss: 0.5038
Iter: 600, Loss: 0.5034
Iter: 700, Loss: 0.5029
Iter: 800, Loss: 0.5025
Iter: 900, Loss: 0.5021
Iter: 1000, Loss: 0.5017
Iter: 1100, Loss: 0.5013
Iter: 1200, Loss: 0.5009
Iter: 1300, Loss: 0.5005
Iter: 1400, Loss: 0.5001
Iter: 1500, Loss: 0.4997
Iter: 1600, Loss: 0.4993
Iter: 1700, Loss: 0.4990
Iter: 1800, Loss: 0.4986
Iter: 1900, Loss: 0.4983
Iter: 2000, Loss: 0.4979
Iter: 2100, Loss: 0.4976
Iter: 2200, Loss: 0.4972
Iter: 2300, Loss: 0.4969
Iter: 2400, Loss: 0.4966
Iter: 2500, Loss: 0.4963
Iter: 2600, Loss: 0.4960
Iter: 2700, Loss: 0.4957
Iter: 2800, Loss: 0.4954
Iter: 2900, Loss: 0.4951
Iter: 3000, Loss: 0.4948
Iter: 3100, Loss: 0.4945
Iter: 3200, Loss: 0.4942
Iter: 3300, Loss: 0.4939
Iter: 3400, Loss: 0.4937
Iter: 3500, Loss: 0.4934
Iter: 3600, Loss: 0.4931
Iter: 3700, Loss: 0.4929
Iter: 3800, Loss: 0.4926
Iter: 3900, Loss: 0.4924
Iter: 4000, 

## 테스트

In [55]:
def accuracy_fn(hypothesis, labels):
    predicted = tf.cast(hypothesis > 0.5, dtype=tf.float32)
    accuracy = tf.reduce_mean(tf.cast(tf.equal(predicted, labels), dtype=tf.float32))
    return accuracy

In [56]:
test_acc = accuracy_fn(logistic_regression(x_test),y_test)
print("Testset Accuracy: {:.4f}".format(test_acc))

Testset Accuracy: 0.7700
